In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
from plotly import graph_objects as go
import plotly.express as px
from matplotlib import rc
%matplotlib inline
from matplotlib import font_manager
plt.style.use('default')
f_path = "C:/windows/Fonts/malgun.ttf"
ko_font = font_manager.FontProperties(fname=f_path).get_name()
rc('font', family='Malgun Gothic')
#plt.title('테스트')

In [3]:
import pandas as pd
import numpy as np

In [151]:
df = pd.read_csv('./data/data_cluster.csv')

In [ ]:
# 데이터 로드 및 'order date (DateOrders)' 컬럼을 datetime 형식으로 변환
df['order date (DateOrders)'] = pd.to_datetime(df['order date (DateOrders)'])

## 시계열 분석

### 고객이 각 클러스터에 언제 신규로 포함되었는가?

In [179]:
# 각 고객별로 첫 구매 날짜와 그 때의 군집을 계산
customer_first_purchase = df.groupby('Customer Id').agg({
    'order date (DateOrders)': 'min',  # 첫 구매 날짜
    'Cluster': 'first'  # 첫 구매 시의 군집
}).reset_index()

# 첫 구매 날짜를 월별로 변환
customer_first_purchase['First Purchase Month'] = customer_first_purchase['order date (DateOrders)'].dt.to_period('M')

# 월별, 군집별로 신규 포함된 고객 수 계산
new_customers_per_cluster = customer_first_purchase.groupby(['First Purchase Month', 'Cluster']).size().reset_index(name='New Customers')

# Period 객체를 YYYY-MM 형식의 문자열로 변환
new_customers_per_cluster['First Purchase Month'] = new_customers_per_cluster['First Purchase Month'].dt.strftime('%Y-%m')

# 2017년 10월 데이터 제외
new_customers_per_cluster = new_customers_per_cluster[new_customers_per_cluster['First Purchase Month'] != '2017-10']

# 그래프 객체 생성
fig = go.Figure()

# 신규 고객 수 데이터 추가 (선 그래프)
for cluster in new_customers_per_cluster['Cluster'].unique():
    cluster_data = new_customers_per_cluster[new_customers_per_cluster['Cluster'] == cluster]
    fig.add_trace(go.Scatter(
        x=cluster_data['First Purchase Month'],
        y=cluster_data['New Customers'],
        mode='lines+markers',
        name=f'{cluster}'
    ))

# 레이아웃 설정
fig.update_layout(
    title='New Customer Count by Cluster Over Time',
    xaxis_title='Month',
    yaxis_title='New Customers',
    legend_title='Cluster'
)

# 그래프 표시
fig.show()

> 데이터 자체에서 언제부터 운영이 시작되었는지는 알 수 없으므로 2016년부터 확인

### 2016년 부터 보기

In [203]:
# 각 고객별로 첫 구매 날짜와 그 때의 군집을 계산
customer_first_purchase = df.groupby('Customer Id').agg({
    'order date (DateOrders)': 'min',  # 첫 구매 날짜
    'Cluster': 'first'  # 첫 구매 시의 군집
}).reset_index()

# 첫 구매 날짜를 월별로 변환
customer_first_purchase['First Purchase Month'] = customer_first_purchase['order date (DateOrders)'].dt.to_period('M')

# 월별, 군집별로 신규 포함된 고객 수 계산
new_customers_per_cluster = customer_first_purchase.groupby(['First Purchase Month', 'Cluster']).size().reset_index(name='New Customers')

# Period 객체를 YYYY-MM 형식의 문자열로 변환
new_customers_per_cluster['First Purchase Month'] = new_customers_per_cluster['First Purchase Month'].dt.strftime('%Y-%m')

# 2016년 이후 및 2017년 10월 제외 데이터 필터링
new_customers_per_cluster = new_customers_per_cluster.query("('2016-01' <= `First Purchase Month`) & (`First Purchase Month` != '2017-10')")

# 그래프 객체 생성
fig = go.Figure()

# 신규 고객 수 데이터 추가 (선 그래프)
for cluster in new_customers_per_cluster['Cluster'].unique():
    cluster_data = new_customers_per_cluster[new_customers_per_cluster['Cluster'] == cluster]
    fig.add_trace(go.Scatter(
        x=cluster_data['First Purchase Month'],
        y=cluster_data['New Customers'],
        mode='lines+markers',
        name=f'Cluster {cluster}'
    ))

# 레이아웃 설정
fig.update_layout(
    title='New Customer Count by Cluster from 2016 Onwards',
    xaxis_title='Month',
    yaxis_title='New Customers',
    legend_title='Cluster'
)

# 그래프 표시
fig.show()

> VIP 군집으로 신규로 포함되는 인원은 거의 없음 => 처음부터 잘 샀던 단골 고객일 가능성이 높음

> 충성고객이 2016년 초에 유입이 컸음을 알 수 있음 => 하지만 2016년 말부터는 거의 유입이 없는 것을 확인. 마케팅 필요

> 이탈 위험 고객은 거의 모든 연도에 가장 높은 값을 가지는 데 이는 찍먹해보는 고객으로 해석됨. 이러한 고객이라도 꾸준히 유입된다면 매출에 어느만큼 영향이 있을 것으로 보임

### 실제 활동 군집별 고객 수

In [204]:
# 월별로 그룹화하고 클러스터별로 고객 수 계산
active_customers_per_month = df.groupby([df['order date (DateOrders)'].dt.to_period('M'), 'Cluster'])['Customer Id'].nunique().reset_index()

# Period 객체를 YYYY-MM 형식의 문자열로 변환
active_customers_per_month['order date (DateOrders)'] = active_customers_per_month['order date (DateOrders)'].dt.strftime('%Y-%m')

# 2017년 10월 데이터 제외
active_customers_per_month = active_customers_per_month[active_customers_per_month['order date (DateOrders)'] != '2017-10']

# 그래프 객체 생성
fig = go.Figure()

# 활동 고객 수 데이터 추가 (선 그래프)
for cluster in active_customers_per_month['Cluster'].unique():
    cluster_data = active_customers_per_month[active_customers_per_month['Cluster'] == cluster]
    fig.add_trace(go.Scatter(
        x=cluster_data['order date (DateOrders)'],
        y=cluster_data['Customer Id'],
        mode='lines+markers',
        name=f'{cluster}'
    ))

# 레이아웃 설정
fig.update_layout(
    title='Active Customer Count by Cluster Per Month',
    xaxis_title='Month',
    yaxis_title='Number of Active Customers',
    legend_title='Cluster'
)

# 그래프 표시
fig.show()

> 2016년 중반부터 불만족 고객이나 이탈위험의 충성 고객이 대거로 활동하지 않음

> 다만 2017년부터 충성고객의 활동은 급격하게 올랐으며 이탈위험고객도 상승세

> 2016년 대거 활동 중단으로 인해 2017년 마케팅을 실시했고 이는 효과적이었음을 암시

> vip 고객은 전반적으로 양호한 숫자를 가짐. 우수한 단골 고객을 유지하는데 좋은 운영방식을 가지고 있음을 나타냄

### 월별 군집별 이탈 고객수

**군집별 구매 간격 평균을 구하고 해당 간격만큼 지났을 때도 구매 이력이 없다면 이탈 고객으로 판단**

In [194]:
# 데이터 프레임 복사 및 초기 처리
df_copy = df.copy()
df_copy['order date (DateOrders)'] = pd.to_datetime(df_copy['order date (DateOrders)'])
df_copy = df_copy.drop_duplicates(subset=['Customer Id', 'order date (DateOrders)', 'Order Id'])

# 각 고객별 구매 날짜 간의 차이 계산, 구매 간격이 0일인 경우 1일로 조정
df_copy['Purchase Interval'] = df_copy.sort_values('order date (DateOrders)').groupby('Customer Id')['order date (DateOrders)'].diff().dt.days
df_copy['Purchase Interval'] = df_copy['Purchase Interval'].replace(0, 1)

# 군집별 평균 구매 간격 계산
cluster_avg_intervals = df_copy.groupby('Cluster')['Purchase Interval'].mean().reset_index()
cluster_avg_intervals.columns = ['Cluster', 'Avg Purchase Interval']
cluster_avg_intervals

,Cluster,Avg Purchase Interval
0,At Risk Customers,279.970144
1,Dissatisfied Customers,166.794637
2,Loyal Customers,173.479396
3,Loyal Customers(At Risk),143.343863
4,VIP Customers,113.353965


In [201]:
cluster_avg_intervals['Avg Purchase Interval'].mean()

175.38840095228812

In [205]:
# 이탈 고객 계산 전처리
customer_last_purchase = df.groupby('Customer Id').agg({
    'order date (DateOrders)': 'max',  # 마지막 거래 날짜
    'Cluster': 'last'  # 마지막 거래 시의 군집
}).reset_index()

# 평균 구매 간격을 반영한 이탈 날짜 계산
customer_last_purchase = pd.merge(customer_last_purchase, cluster_avg_intervals, on='Cluster', how='left')
customer_last_purchase['Churn Date'] = customer_last_purchase['order date (DateOrders)'] + pd.to_timedelta(customer_last_purchase['Avg Purchase Interval'], unit='D')

# 이탈 고객을 다음 월로 정의
customer_last_purchase['Churn Month'] = customer_last_purchase['Churn Date'].dt.to_period('M') + 1

# 2017년 9월 이전 데이터만 포함 (2017년 9월 데이터 포함)
customer_last_purchase = customer_last_purchase[customer_last_purchase['Churn Month'] <= '2017-10']

# 이탈 고객 수 계산
churned_customers_per_month = customer_last_purchase.groupby(['Churn Month', 'Cluster']).size().reset_index(name='Churned Customers')
churned_customers_per_month['Churn Month'] = churned_customers_per_month['Churn Month'].dt.strftime('%Y-%m')

# 그래프 객체 생성
fig = go.Figure()

# 이탈 고객 수 데이터 추가 (선 그래프)
for cluster in churned_customers_per_month['Cluster'].unique():
    cluster_data = churned_customers_per_month[churned_customers_per_month['Cluster'] == cluster]
    fig.add_trace(go.Scatter(
        x=cluster_data['Churn Month'],
        y=cluster_data['Churned Customers'],
        mode='lines+markers',
        name=f'{cluster}'
    ))

# 레이아웃 설정
fig.update_layout(
    title='Churned Customer Count by Cluster Per Month, Considering Cluster-Specific Purchase Intervals',
    xaxis_title='Month',
    yaxis_title='Number of Churned Customers',
    legend_title='Cluster'
)

# 그래프 표시
fig.show()

> 불만족 고객의 이탈은 예상했으나 이탈위험이 있는 충성고객이 2016년 중반부터 대거 이탈을 보여줌

> 또한 2017년부터 충성고객과 VIP 고객 또한 이탈 시작되고 있음

> 하지만 신규 유입은 없는 것으로 파악되지만 실제 활동하는 고객 수는 크게 변동이 없어서 휴먼 고객의 활동이 훨씬 많아졌음을 암시

> 특별 이벤트가 있었을 가능성이 있음

### 월별 주문 빈도

In [206]:

# 월별로 그룹화하고 클러스터별로 주문 수와 고객 수 계산
monthly_stats = df.groupby([df['order date (DateOrders)'].dt.to_period('M'), 'Cluster']).agg({
    'Order Id': 'size',  # 주문 수
    'Customer Id': 'nunique'  # 고객 수
}).reset_index()

# Period 객체를 YYYY-MM 형식의 문자열로 변환
monthly_stats['order date (DateOrders)'] = monthly_stats['order date (DateOrders)'].dt.strftime('%Y-%m')

# 2017년 10월 데이터 제외
monthly_stats = monthly_stats[monthly_stats['order date (DateOrders)'] != '2017-10']

# 그래프 객체 생성
fig = go.Figure()

# 주문 수에 대한 선 그래프 추가
for cluster in monthly_stats['Cluster'].unique():
    cluster_data = monthly_stats[monthly_stats['Cluster'] == cluster]
    fig.add_trace(go.Scatter(
        x=cluster_data['order date (DateOrders)'],
        y=cluster_data['Order Id'],
        mode='lines+markers+text',
        name=f'{cluster}',
        text=cluster_data['Customer Id'],  # 고객 수 표시
        textposition="top center",
        textfont=dict(
            family="Arial, sans-serif",
            size=10,  # 텍스트 크기 조절
            color="black")
        ))

# 그래프 레이아웃 설정
fig.update_layout(
    title='Monthly Orders and Customer Count by Cluster',
    xaxis_title='Order Date',
    yaxis_title='Number of Orders',
    legend_title='Legend'
)

# 그래프 표시
fig.show()

> 충성 고객과 VIP 고객의 주문 빈도가 높음 => 매우 좋은 지표

> 이탈위험 고객도 2017년 부터 주문 빈도가 높아짐

> 2017년에 프로모션이나 이벤트가 진행됐을 가능성

> 좀 더 자세히 보기 위해 실제 활동 고객 수로 나누어 살펴보기

In [207]:
# 월별로 그룹화하고 클러스터별로 주문 수와 고객 수 계산
monthly_stats = df.groupby([df['order date (DateOrders)'].dt.to_period('M'), 'Cluster']).agg({
    'Order Id': 'size',  # 주문 수
    'Customer Id': 'nunique'  # 고객 수
}).reset_index()

# Period 객체를 YYYY-MM 형식의 문자열로 변환
monthly_stats['order date (DateOrders)'] = monthly_stats['order date (DateOrders)'].dt.strftime('%Y-%m')

# 주문 수를 고객 수로 나누어 주문당 고객 수 비율 계산
monthly_stats['Orders per Customer'] = monthly_stats['Order Id'] / monthly_stats['Customer Id']

# 2017년 10월 데이터 제외
monthly_stats = monthly_stats[monthly_stats['order date (DateOrders)'] != '2017-10']

# 그래프 객체 생성
fig = go.Figure()

# 주문당 고객 수 비율에 대한 선 그래프 추가
for cluster in monthly_stats['Cluster'].unique():
    cluster_data = monthly_stats[monthly_stats['Cluster'] == cluster]
    fig.add_trace(go.Scatter(
        x=cluster_data['order date (DateOrders)'],
        y=cluster_data['Orders per Customer'],
        mode='lines+markers',
        name=f'{cluster}'
    ))

# 그래프 레이아웃 설정
fig.update_layout(
    title='Monthly Orders per Customer by Cluster',
    xaxis_title='Order Date',
    yaxis_title='Orders per Customer',
    legend_title='Legend'
)

# 그래프 표시
fig.show()

> VIP 고객과 충성고객은 꾸준히 한명당 주문 수가 높음

> 이탈 위험의 충성고객도 높은 값을 가져서 이탈하지 않도록 프로모션을 진행하거나 설문조사가 필요

### 월별 군집별 실제 수익 분포

In [208]:
# 월별로 그룹화하고 클러스터별로 매출 총합 및 고객 수 계산
monthly_stats = df.groupby([df['order date (DateOrders)'].dt.to_period('M'), 'Cluster']).agg({
    'Sales per customer': 'sum',
    'Customer Id': 'nunique'
}).reset_index()

# Period 객체를 YYYY-MM 형식의 문자열로 변환
monthly_stats['order date (DateOrders)'] = monthly_stats['order date (DateOrders)'].dt.strftime('%Y-%m')

# 2017년 10월 데이터 제외
monthly_stats = monthly_stats[monthly_stats['order date (DateOrders)'] != '2017-10']

# 그래프 초기화
fig = go.Figure()

# 매출 데이터 추가 (선 그래프)
for cluster in monthly_stats['Cluster'].unique():
    filtered_data = monthly_stats[monthly_stats['Cluster'] == cluster]
    fig.add_trace(go.Scatter(
        x=filtered_data['order date (DateOrders)'], 
        y=filtered_data['Sales per customer'],
        mode='lines+markers+text', 
        name=f'{cluster}',
        text=filtered_data['Customer Id'],
        textposition="top center",
        textfont=dict(
            family="Arial, sans-serif",
            size=10,  # 텍스트 크기 조절
            color="black"
        )
    ))

# 레이아웃 설정
fig.update_layout(
    title='Monthly Sales and Customer Count by Cluster',
    xaxis_title='Month',
    yaxis_title='Total Sales',
    legend_title='Legend'
)

# 그래프 표시
fig.show()

> 주문 빈도와 비슷하게 충성고객과 VIP 고객의 실제 수익값이 높음 => 매우 좋은 지표

> 이탈위험 고객도 2017년 부터 수익이 높아짐

> 2017년에 프로모션이나 이벤트가 진행됐을 가능성

> 좀 더 자세히 보기 위해 실제 활동 고객 수로 나누어 살펴보기

In [209]:
# 월별로 그룹화하고 클러스터별로 매출 총합 및 고객 수 계산
monthly_stats = df.groupby([df['order date (DateOrders)'].dt.to_period('M'), 'Cluster']).agg({
    'Sales per customer': 'sum',  # 매출 총합
    'Customer Id': 'nunique'  # 고객 수
}).reset_index()

# Period 객체를 YYYY-MM 형식의 문자열로 변환
monthly_stats['order date (DateOrders)'] = monthly_stats['order date (DateOrders)'].dt.strftime('%Y-%m')

# 매출을 고객 수로 나누어 평균 매출 계산
monthly_stats['Sales per Customer'] = monthly_stats['Sales per customer'] / monthly_stats['Customer Id']

# 2017년 10월 데이터 제외
monthly_stats = monthly_stats[monthly_stats['order date (DateOrders)'] != '2017-10']

# 그래프 초기화
fig = go.Figure()

# 평균 매출 데이터 추가 (선 그래프)
for cluster in monthly_stats['Cluster'].unique():
    filtered_data = monthly_stats[monthly_stats['Cluster'] == cluster]
    fig.add_trace(go.Scatter(
        x=filtered_data['order date (DateOrders)'],
        y=filtered_data['Sales per Customer'],
        mode='lines+markers',  # 텍스트를 제외한 선과 마커 모드
        name=f'{cluster}'
    ))

# 레이아웃 설정
fig.update_layout(
    title='Monthly Sales per Customer by Cluster',
    xaxis_title='Month',
    yaxis_title='Sales per Customer',
    legend_title='Legend'
)

# 그래프 표시
fig.show()

> 2017년도 부터 VIP, 충성고객, 이탈위험 고객 모두 고객당 수익이 높아지고 있음

> 2017년도에서 진행된 프로모션이나 이벤트와 관련된 자료가 있으면 추가적인 분석 가능할 것으로 보임

## CLTV

> 고객이 평생 우리 기업에 어느 정도의 가치를 가져올 수 있을지를 나타내는 지표. 쉽게 말해, 고객 한 명에게 기대할 수 있는 매출과 수익

-  위의 분석에서 군집별로 가치를 어느정도 평가 진행

- 더 논리적이고 확실한 KPI 값으로 진행하기 위해 CLTV 분석 진행

**고객 생애 가치 = 고객 가치 * 평균 고객 수명**


- 고객 가치 (=평균 구매 가치 * 평균 구매 빈도)

- 평균 구매 가치 (=특정 기간 내 총 수익 / 특정 기간 내 총 구매 횟수)
- 평균 구매 빈도 (=특정 기간 내 구매 횟수 / 특정 기간 내 총 고객 수)
- 평균 고객 수명 (=고객이 지속적으로 구매한 기간 / 총 고객 수)

*여기서 평균 고객 수명을 1/평균 구매 간격 으로 계산*

*군집별로 따로 계산 진행*

### 연도별

In [154]:
# 연도 추출
df['Year'] = df['order date (DateOrders)'].dt.year

# 고객별, 연도별 고유 구매 횟수 및 총 매출 계산
customer_sales = df.groupby(['Customer Id', 'Year']).agg({
    'Sales per customer': 'sum',
    'Order Id': 'nunique'
}).rename(columns={
    'Order Id': 'Frequency',
    'Sales per customer': 'MonetaryValue'
})

# 고객별 평균 구매 간격 계산
# 데이터를 'Customer Id', 'Year', 'order date (DateOrders)' 기준으로 정렬
df_sorted = df.sort_values(by=['Customer Id', 'Year', 'order date (DateOrders)'])

# 중복된 'Order Id'가 같은 날짜를 가진 항목을 제거
df_unique_orders = df_sorted.drop_duplicates(subset=['Customer Id', 'Year', 'Order Id'])


# 주문 날짜 간의 평균 간격 계산 후, 0을 1로 대체
purchase_interval = df_unique_orders.groupby(['Customer Id', 'Year'])['order date (DateOrders)'].apply(
    lambda x: np.where(x.diff().dropna().mean().days == 0, 1, x.diff().dropna().mean().days)
)


# 구매 간격이 NaN인 경우 - 이 경우, 연도 내 평균 구매 간격을 사용하거나 다른 논리적인 값으로 대체 가능
# 예를 들어, 해당 연도의 전체 일수(365 또는 366) 또는 반년(182) 등을 고려 가능
# 데이터 프레임으로 변환
purchase_interval = purchase_interval.reset_index(name='Purchase Interval')

# 'Purchase Interval' 열에서 NaN 값이 배열(array)인 경우, 이를 실제 NaN 값으로 변경
purchase_interval['Purchase Interval'] = purchase_interval['Purchase Interval'].apply(lambda x: np.nan if isinstance(x, np.ndarray) and np.isnan(x) else x)

# NaN 값을 365로 변경
purchase_interval['Purchase Interval'] = purchase_interval['Purchase Interval'].fillna(365) # 이는 해당 연도에 단 한 번 구매한 경우 한 해 전체로 간격을 가정

# Purchase Interval 값을 정수형으로 변환
purchase_interval['Purchase Interval'] = purchase_interval['Purchase Interval'].astype(int)


customer_sales.reset_index(inplace=True)


# 데이터 병합
cltv_data = pd.merge(customer_sales, purchase_interval, on=['Customer Id', 'Year'])

# 고객 가치 및 CLTV 계산
cltv_data['Customer Value'] = cltv_data['MonetaryValue'] / cltv_data['Frequency']
cltv_data['CLTV'] = cltv_data['Customer Value'] * cltv_data['Frequency'] / cltv_data['Purchase Interval']

# 클러스터 정보 추가 및 집계
data_unique = df.drop_duplicates(subset=['Customer Id', 'Year'])
cltv_data = cltv_data.merge(data_unique[['Customer Id', 'Year', 'Cluster']], on=['Customer Id', 'Year'], how='left')
average_cltv_by_cluster_year = cltv_data.groupby(['Year', 'Cluster'])['CLTV'].mean().reset_index()

average_cltv_by_cluster_year

    Year                   Cluster       CLTV
0   2015         At Risk Customers   7.318064
1   2015    Dissatisfied Customers  13.451776
2   2015           Loyal Customers  23.473312
3   2015  Loyal Customers(At Risk)  21.451659
4   2015             VIP Customers  33.123908
5   2016         At Risk Customers  10.629012
6   2016    Dissatisfied Customers  16.965278
7   2016           Loyal Customers  21.818773
8   2016  Loyal Customers(At Risk)  23.011130
9   2016             VIP Customers  30.737849
10  2017         At Risk Customers  12.772703
11  2017           Loyal Customers  26.024316
12  2017  Loyal Customers(At Risk)  19.569709
13  2017             VIP Customers  35.940987


In [155]:
# Plotly 시각화
fig = px.line(average_cltv_by_cluster_year, x='Year', y='CLTV', color='Cluster', 
              title='Average CLTV by Cluster Over Years', 
              labels={'CLTV': 'Customer Lifetime Value', 'Year': 'Year', 'Cluster': 'Cluster'},
              markers=True)

# 레이아웃 조정
fig.update_layout(
    xaxis=dict(title='Year', tickmode='linear'),
    yaxis=dict(title='Average CLTV'),
    legend_title_text='Cluster'
)

# 그래프 보기
fig.show()

C:\Users\82102\anaconda3\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



> VIP 고객은 역시 높은 값을 가짐

> 자세히 보기 위해 분기별로 확인해보자

## 분기별

In [162]:
# 연도 및 분기 추출
df['YearQuarter'] = df['order date (DateOrders)'].dt.to_period('Q')

# 고객별, 분기별 고유 구매 횟수 및 총 매출 계산
customer_sales = df.groupby(['Customer Id', 'YearQuarter']).agg({
    'Sales per customer': 'sum',
    'Order Id': 'nunique'
}).rename(columns={
    'Order Id': 'Frequency',
    'Sales per customer': 'MonetaryValue'
})

# 데이터를 'Customer Id', 'YearQuarter', 'order date (DateOrders)' 기준으로 정렬
df_sorted = df.sort_values(by=['Customer Id', 'YearQuarter', 'order date (DateOrders)'])

# 중복된 'Order Id'가 같은 날짜를 가진 항목을 제거
df_unique_orders = df_sorted.drop_duplicates(subset=['Customer Id', 'YearQuarter', 'Order Id'])

# 주문 날짜 간의 평균 간격 계산 후, 0을 1로 대체
purchase_interval = df_unique_orders.groupby(['Customer Id', 'YearQuarter'])['order date (DateOrders)'].apply(
    lambda x: np.where(x.diff().dropna().mean().days == 0, 1, x.diff().dropna().mean().days)
)

# 데이터 프레임으로 변환 및 NaN 처리
purchase_interval = purchase_interval.reset_index(name='Purchase Interval')
purchase_interval['Purchase Interval'] = purchase_interval['Purchase Interval'].apply(lambda x: np.nan if isinstance(x, np.ndarray) and np.isnan(x) else x)
purchase_interval['Purchase Interval'] = purchase_interval['Purchase Interval'].fillna(90) # 분기별 구매가 없는 경우, 세 달 간격을 가정

# 데이터 병합
customer_sales.reset_index(inplace=True)
cltv_data = pd.merge(customer_sales, purchase_interval, on=['Customer Id', 'YearQuarter'])

# 고객 가치 및 CLTV 계산
cltv_data['Customer Value'] = cltv_data['MonetaryValue'] / cltv_data['Frequency']
cltv_data['CLTV'] = cltv_data['Customer Value'] * cltv_data['Frequency'] / cltv_data['Purchase Interval']

# 클러스터 정보 추가 및 집계
data_unique = df.drop_duplicates(subset=['Customer Id', 'YearQuarter'])
cltv_data = cltv_data.merge(data_unique[['Customer Id', 'YearQuarter', 'Cluster']], on=['Customer Id', 'YearQuarter'], how='left')
average_cltv_by_cluster_year_quarter = cltv_data.groupby(['YearQuarter', 'Cluster'])['CLTV'].mean().reset_index()

#print(average_cltv_by_cluster_year_quarter)

In [161]:
# 'YearQuarter' 필드를 문자열 형식으로 변환
average_cltv_by_cluster_year_quarter['YearQuarter'] = average_cltv_by_cluster_year_quarter['YearQuarter'].astype(str)

# 2017Q4 데이터 제외
filtered_data = average_cltv_by_cluster_year_quarter[average_cltv_by_cluster_year_quarter['YearQuarter'] != '2017Q4']

# Plotly 시각화
fig = px.line(filtered_data, x='YearQuarter', y='CLTV', color='Cluster', 
              title='Average CLTV by Cluster Over Quarters', 
              labels={'CLTV': 'Customer Lifetime Value', 'YearQuarter': 'Quarter', 'Cluster': 'Cluster'},
              markers=True)

# 레이아웃 조정
fig.update_layout(
    xaxis=dict(title='Quarter', tickmode='linear', type='category'),  # x축을 카테고리 타입으로 설정
    yaxis=dict(title='Average CLTV'),
    legend_title_text='Cluster'
)

# 그래프 보기
fig.show()

C:\Users\82102\anaconda3\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



> VIP 고객은 꾸준히 높은 값을 가짐

> 2017년도 부터 VIP, 충성고객, 이탈 위험고객이 상승세로 보임. 이들에게 프로모션과 이벤트를 추진하면 좋을 것

> 불만족 고객과 이탈 위험 충성 고객은 이미 떠나갔으므로 아예 신규고객을 유치 위한 마케팅 전략 수립 필요

## 주문 패턴 분석

In [17]:
# 요일과 시간 컬럼 추가
df['order_weekday'] = df['order date (DateOrders)'].dt.weekday
df['order_hour'] = df['order date (DateOrders)'].dt.hour

# 요일을 문자열로 변환
weekday_map = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
df['order_weekday'] = df['order_weekday'].map(weekday_map)

# 군집별 요일별 주문 통계
cluster_weekday_orders = df.groupby(['Cluster', 'order_weekday']).size().reset_index(name='Count')
cluster_weekday_orders = cluster_weekday_orders.pivot(index='order_weekday', columns='Cluster', values='Count').fillna(0).reset_index()
fig_weekday = px.line(cluster_weekday_orders, x='order_weekday', y=cluster_weekday_orders.columns[1:], markers=True,
                      title='Clustered Orders by Weekday',
                      labels={'value': 'Order Count', 'order_weekday': 'Weekday'})
fig_weekday.show()

# 군집별 시간대별 주문 통계
cluster_hourly_orders = df.groupby(['Cluster', 'order_hour']).size().reset_index(name='Count')
cluster_hourly_orders = cluster_hourly_orders.pivot(index='order_hour', columns='Cluster', values='Count').fillna(0).reset_index()
fig_hour = px.line(cluster_hourly_orders, x='order_hour', y=cluster_hourly_orders.columns[1:], markers=True,
                   title='Clustered Orders by Hour',
                   labels={'value': 'Order Count', 'order_hour': 'Hour of the Day'})
fig_hour.show()

> 엇비슷

> 일단 확인 해본거